# 課題の概要

この宿題では、講義で学んだRAG(Retrieval-Augmented Generation)技術を用いて、LLMの生成
内容を改善する実践的な取り組みを行います。演習で利用したコードをベースに、独自の質問と参照文書を用
いて実験を行い、RAGの効果を定量的・定性的に評価します。
この宿題を通じて、「テストデータの作成」と「改善のプロセス」について理解を深め、実際のアプリケーション開発
に役立てることを目指します。



## 内容

宿題の内容
1. 独自の質問と参照資料の作成
- 自分で5つ以上の質問文を考案してください
- 各質問に対する回答を含む参照文書を用意してください
- 少なくとも1つは、LLMが単体では正確に答えられないような知識を含む質問にしてください
2. 実験の実施
- 演習で使用したコードをベースに、以下の2つの方法で回答を生成してください
  - ベースのLLM(RAGなし)での回答生成
  -  RAGを組み合わせた回答生成
- 回答の評価では、単純なYes/No判定でも良いです
  - より詳細な評価指標も検討していただけるとなお良いです

3. 結果分析と考察

- 生成した結果をまとめ、RAGありとRAGなしの差異を分析してください
- RAGによって回答が改善したケースと悪化したケースの両方について考察してください- 結果に基づいて、RAGの有効性と限界についての考察を記述してください

## 扱う条件

1. 使用するモデル

2024年4月リリースのMeta-Llama-3-8B-Instructを使用する


2. 参考文献

・ディズニー公式

・Wikipedia

・[映画『インサイド・ヘッド』あらすじ&ネタバレ！キャラクター、声優キャスト、トリビアも♪
](https://castel.jp/p/4118)






## 質問概要


・ディズニーに関する質問


1. アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？

  解答：オラフ

2. 「アナと雪の女王」でエピソードが展開される国の名前は何ですか？

  解答：アレンデール



3. ディズニー映画「インサイドヘッド」に関して教えて

  期待する解答：

  主人公少女ライリーの頭の中にいるヨロコビ、カナシミ、イカリ、ムカムカ、ビビリ5つの感情が繰り広げる冒険と1人の少女の成長を描いたストーリー

4. ディズニー映画「インサイドヘッド」で現在までに登場している感情のキャラクターを教えて

  解答：
  
  ヨロコビ、カナシミ、イカリ、ムカムカ、ビビリ

  シンパイ、イイナ、ハズカシ、ダリィ

5. ディズニーとピクサーの違いを教えて

  解答の評価ポイント：

  設立年

  共通する点

  例)

  親会社が、ウォルトディズニーカンパニーであることなど

  異なる点

  例)

  名前・コンセプトなど

  この二つが明確にわけられているかどうかを判断基準として定性的に評価する}

In [ ]:
!git remote add origin https://github.com/ommusuc/no-4-ai-enjineering.git
!git branch -M main
!git push -u origin main

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

In [ ]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=400,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

## 1～5に関して質問を行う


### 1 アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 2. 「アナと雪の女王」でエピソードが展開される国の名前は何ですか？

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "「アナと雪の女王」でエピソードが展開される国の名前は何ですか？"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 3.ディズニー映画「インサイドヘッド」に関して教えて

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニー映画「インサイドヘッド」に関して教えて"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 4.ディズニー映画「インサイドヘッド」で現在までに登場している感情のキャラクターを教えて

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニー映画「インサイドヘッド」で現在までに登場している感情のキャラクターを教えて"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 5 ディズニーとピクサーの違いを教えて

In [ ]:
def generate_output2(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=2000,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニーとピクサーの違いを教えて"
response = generate_output(question, system_prompt)

In [ ]:
response